In [1]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torchvision import datasets, transforms
from torch.optim.lr_scheduler import StepLR
import torch.nn.utils.prune as prune
import numpy as np
import matplotlib.pyplot as plt

In [3]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

class LeNet(nn.Module):
    def __init__(self):
        super(LeNet, self).__init__()
        self.conv1 = nn.Conv2d(1, 6, 3)
        self.conv2 = nn.Conv2d(6, 16, 3)
        self.fc1 = nn.Linear(16 * 5 * 5, 120)
        self.fc2 = nn.Linear(120, 84)
        self.fc3 = nn.Linear(84, 10)

    def forward(self, x):
        x = F.max_pool2d(F.relu(self.conv1(x)), (2, 2))
        x = F.max_pool2d(F.relu(self.conv2(x)), 2)
        x = x.view(-1, int(x.nelement() / x.shape[0]))
        x = F.relu(self.fc1(x))
        x = F.relu(self.fc2(x))
        x = F.log_softmax(self.fc3(x))
        return x

model = LeNet().to(device=device)

In [5]:
init_state = dict(model.state_dict())

In [6]:
train_loader = torch.utils.data.DataLoader(
    datasets.MNIST('../data', train=True, download=True, transform=transforms.Compose([
    transforms.ToTensor(),])),batch_size=16, shuffle=True)

In [7]:
def train(model, device, train_loader, optimizer, epoch):
    model.train()
    for batch_idx, (data, target) in enumerate(train_loader):
        data, target = data.to(device), target.to(device)
        optimizer.zero_grad()
        output = model(data)
        loss = F.nll_loss(output, target)
        loss.backward()
        optimizer.step()
        if batch_idx % 10000 == 0:
            print('Train Epoch: {} [{}/{} ({:.0f}%)]\tLoss: {:.6f}'.format(
                epoch, batch_idx * len(data), len(train_loader.dataset),
                100. * batch_idx / len(train_loader), loss.item()))

In [8]:
def test(model, device, test_loader):
    model.eval()
    test_loss = 0
    correct = 0
    with torch.no_grad():
        for data, target in test_loader:
            data, target = data.to(device), target.to(device)
            output = model(data)
            test_loss += F.nll_loss(output, target, reduction='sum').item()  # sum up batch loss
            pred = output.argmax(dim=1, keepdim=True)  # get the index of the max log-probability
            correct += pred.eq(target.view_as(pred)).sum().item()

    test_loss /= len(test_loader.dataset)

    print('\nTest set: Average loss: {:.4f}, Accuracy: {}/{} ({:.0f}%)\n'.format(
        test_loss, correct, len(test_loader.dataset),
        100. * correct / len(test_loader.dataset)))

In [9]:
use_cuda = True

torch.manual_seed(1)

device = torch.device("cuda" if use_cuda else "cpu")

train_kwargs = {'batch_size': 64}
test_kwargs = {'batch_size': 64}
if use_cuda:
    cuda_kwargs = {'num_workers': 1,
                       'pin_memory': True,
                       'shuffle': True}
    train_kwargs.update(cuda_kwargs)
    test_kwargs.update(cuda_kwargs)

transform=transforms.Compose([
    transforms.ToTensor(),
    transforms.Normalize((0.1307,), (0.3081,))
    ])
dataset1 = datasets.MNIST('../data', train=True, download=True,
                       transform=transform)
dataset2 = datasets.MNIST('../data', train=False,
                       transform=transform)
train_loader = torch.utils.data.DataLoader(dataset1,**train_kwargs)
test_loader = torch.utils.data.DataLoader(dataset2, **test_kwargs)

model  =LeNet().to(device=device)
optimizer = optim.Adam(model.parameters(), lr=0.001)

scheduler = StepLR(optimizer, step_size=1, gamma=0.9)
for epoch in range(0, 5):
    train(model, device, train_loader, optimizer, epoch)
    test(model, device, test_loader)
    scheduler.step()

C:\ProgramData\Anaconda3\envs\tf\lib\site-packages\ipykernel_launcher.py:18: UserWarning: Implicit dimension choice for log_softmax has been deprecated. Change the call to include dim=X as an argument.


Train Epoch: 0 [0/60000 (0%)]	Loss: 2.308216

Test set: Average loss: 0.0873, Accuracy: 9710/10000 (97%)

Train Epoch: 1 [0/60000 (0%)]	Loss: 0.058286

Test set: Average loss: 0.0538, Accuracy: 9820/10000 (98%)

Train Epoch: 2 [0/60000 (0%)]	Loss: 0.072566

Test set: Average loss: 0.0416, Accuracy: 9865/10000 (99%)

Train Epoch: 3 [0/60000 (0%)]	Loss: 0.045023

Test set: Average loss: 0.0385, Accuracy: 9872/10000 (99%)

Train Epoch: 4 [0/60000 (0%)]	Loss: 0.043145

Test set: Average loss: 0.0364, Accuracy: 9884/10000 (99%)



In [10]:
parameters_to_prune = ((model.fc1, 'weight'),(model.fc2, 'weight'),(model.fc3, 'weight'))

prune.global_unstructured(
    parameters_to_prune,
    pruning_method=prune.L1Unstructured,
    amount=0.20)

In [11]:
test(model, device, test_loader)

C:\ProgramData\Anaconda3\envs\tf\lib\site-packages\ipykernel_launcher.py:18: UserWarning: Implicit dimension choice for log_softmax has been deprecated. Change the call to include dim=X as an argument.



Test set: Average loss: 0.0369, Accuracy: 9885/10000 (99%)



In [12]:
prune_state = dict(model.state_dict())

In [13]:
prune_state['fc1.weight_orig'] = init_state['fc1.weight']
prune_state['fc2.weight_orig'] = init_state['fc2.weight']
prune_state['fc3.weight_orig'] = init_state['fc3.weight']

In [14]:
model.load_state_dict(prune_state)

<All keys matched successfully>

In [15]:
test(model, device, test_loader)

C:\ProgramData\Anaconda3\envs\tf\lib\site-packages\ipykernel_launcher.py:18: UserWarning: Implicit dimension choice for log_softmax has been deprecated. Change the call to include dim=X as an argument.



Test set: Average loss: 2.3138, Accuracy: 1014/10000 (10%)



In [16]:
torch.manual_seed(1)

device = torch.device("cuda" if use_cuda else "cpu")

train_kwargs = {'batch_size': 64}
test_kwargs = {'batch_size': 64}
if use_cuda:
    cuda_kwargs = {'num_workers': 1,
                       'pin_memory': True,
                       'shuffle': True}
    train_kwargs.update(cuda_kwargs)
    test_kwargs.update(cuda_kwargs)

transform=transforms.Compose([
    transforms.ToTensor(),
    transforms.Normalize((0.1307,), (0.3081,))
    ])
dataset1 = datasets.MNIST('../data', train=True, download=True,
                       transform=transform)
dataset2 = datasets.MNIST('../data', train=False,
                       transform=transform)
train_loader = torch.utils.data.DataLoader(dataset1,**train_kwargs)
test_loader = torch.utils.data.DataLoader(dataset2, **test_kwargs)

optimizer = optim.Adam(model.parameters(), lr=0.001)

scheduler = StepLR(optimizer, step_size=1, gamma=0.9)
for epoch in range(0, 5):
    train(model, device, train_loader, optimizer, epoch)
    test(model, device, test_loader)
    scheduler.step()

C:\ProgramData\Anaconda3\envs\tf\lib\site-packages\ipykernel_launcher.py:18: UserWarning: Implicit dimension choice for log_softmax has been deprecated. Change the call to include dim=X as an argument.


Train Epoch: 0 [0/60000 (0%)]	Loss: 2.323035

Test set: Average loss: 0.0550, Accuracy: 9812/10000 (98%)

Train Epoch: 1 [0/60000 (0%)]	Loss: 0.049987

Test set: Average loss: 0.0418, Accuracy: 9860/10000 (99%)

Train Epoch: 2 [0/60000 (0%)]	Loss: 0.014573

Test set: Average loss: 0.0381, Accuracy: 9875/10000 (99%)

Train Epoch: 3 [0/60000 (0%)]	Loss: 0.061576

Test set: Average loss: 0.0313, Accuracy: 9900/10000 (99%)

Train Epoch: 4 [0/60000 (0%)]	Loss: 0.028077

Test set: Average loss: 0.0347, Accuracy: 9900/10000 (99%)



In [18]:
for i in range(3):
    parameters_to_prune = ((model.fc1, 'weight'),(model.fc2, 'weight'),(model.fc3, 'weight'))

    prune.global_unstructured(
        parameters_to_prune,
        pruning_method=prune.L1Unstructured,
        amount=0.20)
    prune_state = dict(model.state_dict())

    prune_state['fc1.weight_orig'] = init_state['fc1.weight']
    prune_state['fc2.weight_orig'] = init_state['fc2.weight']
    prune_state['fc3.weight_orig'] = init_state['fc3.weight']
    model.load_state_dict(prune_state)
    test(model, device, test_loader)

    torch.manual_seed(1)

    device = torch.device("cuda" if use_cuda else "cpu")

    train_kwargs = {'batch_size': 64}
    test_kwargs = {'batch_size': 64}
    if use_cuda:
        cuda_kwargs = {'num_workers': 1,
                           'pin_memory': True,
                           'shuffle': True}
        train_kwargs.update(cuda_kwargs)
        test_kwargs.update(cuda_kwargs)

    transform=transforms.Compose([
        transforms.ToTensor(),
        transforms.Normalize((0.1307,), (0.3081,))
        ])
    dataset1 = datasets.MNIST('../data', train=True, download=True,
                           transform=transform)
    dataset2 = datasets.MNIST('../data', train=False,
                           transform=transform)
    train_loader = torch.utils.data.DataLoader(dataset1,**train_kwargs)
    test_loader = torch.utils.data.DataLoader(dataset2, **test_kwargs)

    optimizer = optim.Adam(model.parameters(), lr=0.001)

    scheduler = StepLR(optimizer, step_size=1, gamma=0.9)
    for epoch in range(0, 5):
        train(model, device, train_loader, optimizer, epoch)
        test(model, device, test_loader)
        scheduler.step()

C:\ProgramData\Anaconda3\envs\tf\lib\site-packages\ipykernel_launcher.py:18: UserWarning: Implicit dimension choice for log_softmax has been deprecated. Change the call to include dim=X as an argument.



Test set: Average loss: 2.1479, Accuracy: 1756/10000 (18%)

Train Epoch: 0 [0/60000 (0%)]	Loss: 2.137884

Test set: Average loss: 0.0427, Accuracy: 9861/10000 (99%)

Train Epoch: 1 [0/60000 (0%)]	Loss: 0.045292

Test set: Average loss: 0.0418, Accuracy: 9860/10000 (99%)

Train Epoch: 2 [0/60000 (0%)]	Loss: 0.016013

Test set: Average loss: 0.0359, Accuracy: 9885/10000 (99%)

Train Epoch: 3 [0/60000 (0%)]	Loss: 0.025743

Test set: Average loss: 0.0307, Accuracy: 9903/10000 (99%)

Train Epoch: 4 [0/60000 (0%)]	Loss: 0.019313

Test set: Average loss: 0.0332, Accuracy: 9906/10000 (99%)


Test set: Average loss: 1.8922, Accuracy: 4066/10000 (41%)

Train Epoch: 0 [0/60000 (0%)]	Loss: 1.873355

Test set: Average loss: 0.0389, Accuracy: 9869/10000 (99%)

Train Epoch: 1 [0/60000 (0%)]	Loss: 0.036956

Test set: Average loss: 0.0339, Accuracy: 9897/10000 (99%)

Train Epoch: 2 [0/60000 (0%)]	Loss: 0.005823

Test set: Average loss: 0.0322, Accuracy: 9893/10000 (99%)

Train Epoch: 3 [0/60000 (0%)]	

In [19]:
prune.remove(model.fc1, 'weight')
prune.remove(model.fc2, 'weight')
prune.remove(model.fc3, 'weight')

Linear(in_features=84, out_features=10, bias=True)

In [20]:
test(model, device, test_loader)

C:\ProgramData\Anaconda3\envs\tf\lib\site-packages\ipykernel_launcher.py:18: UserWarning: Implicit dimension choice for log_softmax has been deprecated. Change the call to include dim=X as an argument.



Test set: Average loss: 0.0333, Accuracy: 9916/10000 (99%)



In [32]:
print(
    "Sparsity in fc1.weight: {:.2f}%".format(
        100. * float(torch.sum(model.fc1.weight == 0))
        / float(model.fc1.weight.nelement())
    )
)
print(
    "Sparsity in fc2.weight: {:.2f}%".format(
        100. * float(torch.sum(model.fc2.weight == 0))
        / float(model.fc2.weight.nelement())
    )
)
print(
    "Sparsity in fc3.weight: {:.2f}%".format(
        100. * float(torch.sum(model.fc3.weight == 0))
        / float(model.fc3.weight.nelement())
    )
)
print(
    "Global sparsity: {:.2f}%".format(
        100. * float(
            torch.sum(model.fc1.weight == 0)
            + torch.sum(model.fc2.weight == 0)
            + torch.sum(model.fc3.weight == 0)
        )
        / float(
            model.fc1.weight.nelement()
            + model.fc2.weight.nelement()
            + model.fc3.weight.nelement()
        )
    )
)

Sparsity in fc1.weight: 62.77%
Sparsity in fc2.weight: 43.69%
Sparsity in fc3.weight: 29.88%
Global sparsity: 59.04%


In [33]:
1- (0.8**4)

0.5903999999999999